Copy code into notebook for testing

In [8]:
# https://towardsdatascience.com/getting-weather-data-in-3-easy-steps-8dc10cc5c859
# https://www.ncdc.noaa.gov/cdo-web/webservices/v2#data
import sys
import requests
import pandas as pd
import json
from datetime import datetime,date, timedelta

print('Precipitation Data Pulling...')

Token = 'ziLDJecqBlvTSNGOzOmPPLmBZtXSfBwR'

referencedata = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/RiverReferenceTable.csv')
referencedata['ZipCode2'] = referencedata['ZipCode'].fillna(0).astype(int)
today = date.today()
Dateslist = [today - timedelta(days = day) for day in range(7)]
startdate = min(Dateslist)
enddate = max(Dateslist)
print(startdate,enddate)
all_data = pd.read_csv('CurrentPrecip.csv',index_col=0).drop_duplicates()

for row in range(0,len(referencedata)):
    print(referencedata.Name[row],' Zip Codes')
    name = referencedata.Name[row]
    zipcode = referencedata.ZipCode[row]
    zipcode2 = referencedata.ZipCode2[row]
    print(zipcode,' ',zipcode2)


    data_url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&units=metric&limit=1000&locationid=ZIP:'+str(zipcode)+'&startdate='+str(startdate)+'&enddate='+str(enddate)
    
    r = requests.get(data_url, headers={'token':Token})
    try: 
        if r.status_code == 200:
            d = json.loads(r.text)
            if len(d) == 0:
                data_url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&locationid=ZIP:'+str(zipcode2)+'&startdate='+str(startdate)+'&enddate='+str(enddate)
                r = requests.get(data_url, headers={'token':Token})
                d = json.loads(r.text)
            dates = []
            precips = []
              #get all items in the response which are precipitation readings
            try:
                precip = [item for item in d['results'] if item['datatype']=='PRCP']
                precips += [item['value'] for item in precip]
                dates += [item['date'] for item in precip]
            except KeyError:
                print('No data available at this Zip Code: ',zipcode2)
                continue

            df = pd.DataFrame()
            df['Date'] = [datetime.strptime(d, '%Y-%m-%dT%H:%M:%S') for d in dates]
            if len(precips) > 0:
                df['Precip'] = [v for v in precips]

            df['Name'] = name
            all_data = all_data.append(df,ignore_index=True)
    except:
        KeyError
        continue
    
    


  
# print(all_data)
all_data.to_csv('C:/Users/Scott/Desktop/Projects/River_Data/Ingestion/CurrentPrecip.csv')

Precipitation Data Pulling...
2023-03-27 2023-04-02
Russian River HaciendaBridge  Zip Codes
95436   95436
Navarro River  Zip Codes
95432   95432
Noyo River  Zip Codes
95437   95437
San Lorenzo River Upper  Zip Codes
95018   95018
Mad River  Zip Codes
95573   95573
Putah Creek  Zip Codes
95694   95694
Truckee River Farad  Zip Codes
96161   96161
Little Truckee River  Zip Codes
96161   96161
Smith River CC  Zip Codes
95531   95531
Smith River Fort Dick  Zip Codes
95531   95531
Eel River Fort Seward  Zip Codes
95542   95542
Eel River Scotia  Zip Codes
95565   95565
Eel River Fernbridge  Zip Codes
95540   95540
San Lorenzo River  Zip Codes
95018   95018
Hat Creek  Zip Codes
96013   96013
Yuba River  Zip Codes
95944   95944
North Fork American River  Zip Codes
95603   95603
Pit River  Zip Codes
96051   96051
East Walker  Zip Codes
93517   93517
East Fork Carson  Zip Codes
96120   96120
Trinity River Lewiston  Zip Codes
96052   96052
Trinity River Junction City  Zip Codes
96048   96048
Rogue

In [ ]:
import re
import pandas as pd
import numpy as np
import requests
import json
import time
import datetime as dt

# Functions for Rain types
def LightRain(s):
    if re.search('Slight Chance Rain',s):
        value = 1
    elif re.search('Light Rain',s):
        value = 1  
    else: value = 0
    return value

def Rain(s):
    if re.search('Rain',s): 
        value = 1
    elif re.search('precipitation',s): 
        value = 1
    else: value = 0
    return value

def HeavyRain(s):
    if re.search('Heavy Rain',s):
        value = 1
    else: value = 0
    return value

def Snow(s):
    if re.search('Snow',s):
        value = 1
    else: value = 0
    return value

def WindTranslate(s):
    s = ' '+s
    l = len(s)-4
    value = s[l-2:l]
    return int(value)

def RainCoder(s):
    if s[2] == '1':
        x = 3
    elif s[0] == '1':
        x = 1
    elif s[1] == '1':
        x = 2
    else: x = 0
    return x

# Generalize name options to Today or a weekday
def nameChanger(x):
    if x == 'Sunday Night':
        x = 'Sunday'
    elif x == 'Monday Night':
        x = 'Monday'
    elif x == 'Tuesday Night':
        x = 'Tuesday'
    elif x == 'Wednesday Night':
        x = 'Wednesday'
    elif x == 'Thursday Night':
        x = 'Thursday'
    elif x == 'Friday Night':
        x = 'Friday'
    elif x == 'Saturday Night':
        x = 'Saturday'
    elif x == 'This Afternoon':
        x = 'Today'
    elif x == 'Tonight':
        x = 'Today'

    else: x
    return x

    
print('Forecast Rain Pulling...')
# Parameters
rivers_reference = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/RiverReferenceTable.csv')

sites = rivers_reference.USGS_ID.tolist()
string = ''

for i in sites:
    string = string+','+str(i)
sites=string[1:]

#sites=sites[0:1]

# URL to load json of data 
# Use format=rmd for readable URL
link = ('https://waterservices.usgs.gov/nwis/iv/?format=json&sites='+sites)
r=requests.get(link)
Geodata = json.loads(r.text)

Forecast_df = pd.DataFrame()

# Gather information from API
for i in range(len(Geodata['value']['timeSeries'])):
    lat = str(Geodata['value']['timeSeries'][i]['sourceInfo']['geoLocation']['geogLocation']['latitude'])
    long = str(Geodata['value']['timeSeries'][i]['sourceInfo']['geoLocation']['geogLocation']['longitude'])
    baseStation = Geodata['value']['timeSeries'][i]['sourceInfo']['siteName']
    # Search API for Lat Long Position
    r = requests.get('https://api.weather.gov/points/'+lat+','+long)
    json_data = r.json()

    print('Retrieving '+baseStation)
    # Retrieve Forecast for Lat Long Position
    forecast_json = requests.get(json_data['properties']['forecast']).json()
    try: 
        if forecast_json['status'] == 500:
            continue
        if forecast_json['status'] == 503:
            continue
    except:
        KeyError
    print(forecast_json)
    forecast = forecast_json['properties']['periods']
    Forecast_df_temp = pd.DataFrame(forecast)

    # Clean Data
    Forecast_df_temp['startTime'] = pd.to_datetime(Forecast_df_temp.startTime,utc=True).dt.date
    Forecast_df_temp['endTime'] = pd.to_datetime(Forecast_df_temp.endTime,utc=True).dt.date
    Forecast_df_temp['Lat'] = lat
    Forecast_df_temp['Long'] = long
    Forecast_df_temp['Station'] = baseStation
    Forecast_df_temp['WindMph'] = Forecast_df_temp.windSpeed.apply(lambda x: WindTranslate(x))

    # Generalize Name's of Day
    Forecast_df_temp['WeekdayName'] = Forecast_df_temp.name.apply(lambda x: nameChanger(x))
  
    # Create Rain and Snow  Features
    Forecast_df_temp['LightRainFlag'] = Forecast_df_temp.shortForecast.apply(lambda x: LightRain(x))
    Forecast_df_temp['HeavyRainFlag'] = Forecast_df_temp.shortForecast.apply(lambda x: HeavyRain(x))
    Forecast_df_temp['RainFlag'] = Forecast_df_temp.shortForecast.apply(lambda x: Rain(x))
    Forecast_df_temp['SnowFlag'] = Forecast_df_temp.shortForecast.apply(lambda x: HeavyRain(x))
    Forecast_df_temp['RainCode'] = Forecast_df_temp.LightRainFlag.astype(str) + Forecast_df_temp.RainFlag.astype(str) + Forecast_df_temp.HeavyRainFlag.astype(str)
    Forecast_df_temp['Rain'] = Forecast_df_temp.RainCode.apply(lambda x: RainCoder(x))
    
    Forecast_df_temp = Forecast_df_temp[['Station','number','WeekdayName','startTime','endTime'
                                        ,'Rain','SnowFlag','WindMph','windDirection','temperature'
                                        ,'LightRainFlag','RainFlag','HeavyRainFlag','RainCode','shortForecast','detailedForecast','Lat','Long']]

    Forecast_df = Forecast_df.append(Forecast_df_temp,ignore_index=True)
    
    # # Remove Duplicates
    # Forecast_df = Forecast_df.drop_duplicates()


# Transform Data for Loading    
Weather_Forecast = Forecast_df.set_index('Station').join(rivers_reference[['USGS Name','Name']].set_index('USGS Name'))
Weather_Forecast = Weather_Forecast.reset_index().rename(columns={'index':'Station','Name':'StationName'})
Weather_Forecast['Date_Name'] = Weather_Forecast.startTime.astype(str)+'_'+Weather_Forecast.StationName
Weather_Forecast['ForecastFromPresent'] = Weather_Forecast['number']
Weather_Forecast = Weather_Forecast[['StationName','ForecastFromPresent','WeekdayName','startTime','endTime'
                ,'Rain','SnowFlag','WindMph','windDirection','temperature'
                ,'LightRainFlag','RainFlag','HeavyRainFlag','RainCode','shortForecast','detailedForecast','Lat','Long','Date_Name']]

Avgs = Weather_Forecast.groupby('Date_Name').mean()[['Rain','WindMph','temperature','SnowFlag']]
StaticFields = Weather_Forecast.groupby('Date_Name').min()[['ForecastFromPresent','StationName','WeekdayName','startTime','endTime','windDirection']]

Condensed_df = StaticFields.join(Avgs).reset_index().drop_duplicates(subset=['Date_Name'])

# Condensed_df.to_csv('NewWeatherForecast.csv',index=False)

full_df = pd.read_csv('NewWeatherForecast.csv')
full_df.append(Condensed_df).drop_duplicates()[['Date_Name','ForecastFromPresent','StationName','WeekdayName','startTime','endTime','windDirection','Rain','WindMph','temperature','SnowFlag']].to_csv('NewWeatherForecast.csv',index=True)



#OLD - Nov22
# import re
# import pandas as pd
# import numpy as np
# import requests
# import json
# import time
# import datetime as dt

# print('Forecast Rain Pulling...')
# # Parameters
# rivers_reference = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/RiverReferenceTable.csv')

# sites = rivers_reference.USGS_ID.tolist()
# string = ''

# for i in sites:
#     string = string+','+str(i)
# sites=string[1:]

# # URL to load json of data 
# # Use format=rmd for readable URL
# link = ('https://waterservices.usgs.gov/nwis/iv/?format=json&sites='+sites)
# r=requests.get(link)
# Geodata = json.loads(r.text)

# Forecast_df = pd.DataFrame()

# # Gather information from API
# for i in range(len(Geodata['value']['timeSeries'])):
#     lat = str(Geodata['value']['timeSeries'][i]['sourceInfo']['geoLocation']['geogLocation']['latitude'])
#     long = str(Geodata['value']['timeSeries'][i]['sourceInfo']['geoLocation']['geogLocation']['longitude'])
#     baseStation = Geodata['value']['timeSeries'][i]['sourceInfo']['siteName']
#     # Search API for Lat Long Position
#     r = requests.get('https://api.weather.gov/points/'+lat+','+long)
#     json_data = r.json()

#     # Retrieve Forecast for Lat Long Position
#     forecast_json = requests.get(json_data['properties']['forecast']).json()
#     try: 
#         if forecast_json['status'] == 500:
#             continue
#     except:
#         KeyError

#     forecast = forecast_json['properties']['periods']
#     Forecast_df_temp = pd.DataFrame(forecast).drop(columns='number')
#     Forecast_df_temp['Lat'] = lat
#     Forecast_df_temp['Long'] = long

#     Forecast_df_temp['Station'] = baseStation
#     Forecast_df = Forecast_df.append(Forecast_df_temp,ignore_index=True)

#     # Cleaning DataFrame
#     df = Forecast_df.drop_duplicates()
#     df.startTime = pd.to_datetime(df.startTime,utc=True)
#     df.endTime = pd.to_datetime(df.endTime,utc=True)
#     df.drop(columns=['temperatureTrend','icon'],inplace=True)



# # Functions for Rain types
# def LightRain(s):
#     if re.search('Slight Chance Rain',s):
#         value = 1
#     elif re.search('Light Rain',s):
#         value = 1  
#     else: value = 0
#     return value

# def Rain(s):
#     if re.search('Rain Showers',s):
#         value = 1
#     else: value = 0
#     return value

# def HeavyRain(s):
#     if re.search('Heavy Rain',s):
#         value = 1
#     else: value = 0
#     return value

# def WindTranslate(s):
#     s = ' '+s
#     l = len(s)-4
#     value = s[l-2:l]
#     return int(value)
    
# # Create Rain Boolean Features
# df['LightRain'] = df.shortForecast.apply(lambda x: LightRain(x))
# df['Rain'] = df.shortForecast.apply(lambda x: Rain(x))
# df['HeavyRain'] = df.shortForecast.apply(lambda x: HeavyRain(x))
# df['WindMph'] = df.windSpeed.apply(lambda x: WindTranslate(x))

# # Create final Dataframe with Key
# WeatherForecast = df[['Station','name','startTime','endTime','temperature','windDirection','WindMph','LightRain','Rain','HeavyRain']]
# WeatherForecast = WeatherForecast.set_index('Station').join(rivers_reference[['USGS Name','Name']].set_index('USGS Name'))
# WeatherForecast = WeatherForecast.reset_index().rename(columns={'index':'Station','Name':'StationName'})
# WeatherForecast['Date_Name'] = WeatherForecast.startTime.dt.date.astype(str)+'_'+WeatherForecast.StationName

# # WeatherForecast.to_csv('WeatherForecast.csv',index=False)
# full_df = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/Exploration/WeatherForecast.csv')
# full_df.append(WeatherForecast).drop_duplicates()[['Station','name','startTime','endTime','temperature','windDirection','WindMph','LightRain','Rain','HeavyRain','StationName','Date_Name']].to_csv('WeatherForecast.csv')

Forecast Rain Pulling...
Retrieving E WALKER RV NR BRIDGEPORT, CA
Retrieving E WALKER RV NR BRIDGEPORT, CA
Retrieving E FK CARSON RV BLW MARKLEEVILLE CK NR MARKLEEVILLE
{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld', {'@version': '1.1', 'wx': 'https://api.weather.gov/ontology#', 'geo': 'http://www.opengis.net/ont/geosparql#', 'unit': 'http://codes.wmo.int/common/unit/', '@vocab': 'https://api.weather.gov/ontology#'}], 'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-119.7804894, 38.7293241], [-119.77532749999999, 38.7075335], [-119.7473577, 38.711564599999996], [-119.7525141, 38.7333557], [-119.7804894, 38.7293241]]]}, 'properties': {'updated': '2023-03-22T03:54:58+00:00', 'units': 'us', 'forecastGenerator': 'BaselineForecastGenerator', 'generatedAt': '2023-03-22T05:06:54+00:00', 'updateTime': '2023-03-22T03:54:58+00:00', 'validTimes': '2023-03-21T21:00:00+00:00/P7DT7H', 'elevation': {'unitCode': 'wmoUnit:m', 'value': 1685.8488}, 'periods': [